In [ ]:
from tqdm import tqdm
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_parquet("train-00005-of-00006-fe1acc5d10a9f0e2.parquet")

In [ ]:
data

In [ ]:
data["openai_moderation"][0]

In [ ]:
for key, value in data.openai_moderation.values[0][0]['categories'].items():
    print(value)

In [ ]:
def separate_conv(conv):
    try:
        user_texts = [x['content'] for x in conv if x['role'] == 'user']
        assistant_texts = [x['content'] for x in conv if x['role'] == 'assistant']
        assert len(user_texts) == len(assistant_texts)
        return user_texts, assistant_texts
    except:
        print(conv)
        
data['prompt'], data['response'] = zip(*data.conversation.apply(separate_conv))

In [ ]:
def separate_categories(data):
    try:
        c = data[0]['categories']
        categories = 'Good'
        for key, value in c.items():
            if value == 1:
                return key
        return categories
    except:
        print(data)
        
data['categories']= data.openai_moderation.apply(separate_categories)

In [ ]:
data

In [ ]:
data = data.loc[data.language == 'English'].reset_index(drop=True)

In [ ]:
data

In [ ]:
e_data = data.explode(['prompt','response']).reset_index(drop=True)
all_prompt_list = e_data.prompt.unique().tolist()

In [ ]:
e_data

In [ ]:
len(all_prompt_list)

In [ ]:
all_prompt_list[0]

In [ ]:
ex_33 = pd.read_csv("lmsys-chatbot_arena_conversations-33k_clean.csv")
explode_33 = ex_33.explode(['prompt', 'response_a', 'response_b']).reset_index(drop = True)

In [ ]:
explode_33

In [ ]:
explode_33.prompt.values.tolist()[0], type(explode_33.prompt.values.tolist()[0])

In [ ]:
explode_33.prompt.values.tolist()[0][2:-2]

In [ ]:
explode_33["prompt"] = explode_33["prompt"].apply(lambda x: x[2:-2])

In [ ]:
explode_33["prompt"]

In [ ]:
explode_33.prompt.values.tolist()[1]

In [ ]:
#都出现过的prompt
same_prompt = [i for idx, i in enumerate(all_prompt_list) if i in explode_33.prompt.values.tolist()]
len(same_prompt)

In [ ]:
#获取对应的id，提取数据
con_id = e_data.loc[e_data.prompt.isin(same_prompt), 'conversation_id'].unique().tolist()

In [ ]:
#有着相同prompt的数据
final = data.loc[data.conversation_id.isin(con_id)].reset_index(drop=True)

In [ ]:
prompt_count = final.prompt.value_counts().reset_index()

In [ ]:
prompt_count

In [ ]:
idx = (prompt_count['count'] <= 36) & (prompt_count['count'] >= 2)
prompt_count_more_than = prompt_count.loc[idx].reset_index(drop=True)
prompt_count_more_than

In [ ]:
use_prompt_list = prompt_count_more_than.prompt.tolist()
use_data = final.loc[final.prompt.isin(use_prompt_list)].reset_index(drop=True)
use_data

In [ ]:
use_data["prompt"][0], type(use_data["prompt"][0])

In [ ]:
use_prompt_list[0], type(use_prompt_list[0])

In [ ]:
len(use_prompt_list)

In [ ]:
use_data["categories"].value_counts()

In [ ]:
new_data = pd.DataFrame(columns=['id', 'prompt', 'model_a', 'model_b', 'response_a', 'response_b', 'categories'])

for use_prompt in tqdm(use_prompt_list, total=len(use_prompt_list)):
    temp_use_data = use_data[use_data["prompt"].apply(lambda x: x == use_prompt)]
    temp_use_data.reset_index(drop=True, inplace=True)
    temp_use_data.loc[:, "tuple_response"] = temp_use_data.loc[:, "response"].apply(tuple)
    temp_use_data.drop_duplicates(subset=["model", "tuple_response"])
    for first_idx, first_row in temp_use_data.iterrows():
        for second_idx, second_row in temp_use_data.iterrows():
            if second_idx <= first_idx:
                continue
            else:
                if first_row["categories"] == "Good" and second_row["categories"] == "Good":
                    category = "Both Good"
                elif first_row["categories"] != "Good" and second_row["categories"] == "Good":
                    category = "First Bad"
                elif first_row["categories"] == "Good" and second_row["categories"] != "Good":
                    category = "Second Bad"
                elif first_row["categories"] != "Good" and second_row["categories"] != "Good":
                    category = "Both Bad"
                new_row = {
                    "id": 1,
                    "prompt": second_row["prompt"],
                    "model_a": first_row["model"],
                    "model_b": second_row["model"],
                    "response_a": first_row["response"],
                    "response_b": second_row["response"],
                    "categories": category
                }
                new_row_df = pd.DataFrame([new_row], columns=new_data.columns)
                new_data = pd.concat([new_data, new_row_df], ignore_index=True)

new_data.head()

In [ ]:
new_data.shape

In [ ]:
new_data["id"] = range(len(new_data))
new_data.head()

In [ ]:
new_data["response_a"][0], type(new_data["response_a"][0])

In [ ]:
json_string = json.dumps(new_data["response_a"][0])
print(json_string)
parsed_list = json.loads(json_string)
print(parsed_list)

In [ ]:
def ensure_json_loads(input_list):
    return json.dumps(input_list)

In [ ]:
new_data.loc[:, "prompt"] = new_data["prompt"].apply(ensure_json_loads)
new_data.loc[:, "response_a"] = new_data["response_a"].apply(ensure_json_loads)
new_data.loc[:, "response_b"] = new_data["response_b"].apply(ensure_json_loads)
new_data.head()

In [ ]:
new_data["response_a"][0], type(new_data["response_a"][0])

In [ ]:
new_data.to_parquet("Split6.parquet", index=False)

In [ ]:
pd.read_parquet("Split6.parquet")